In [158]:
import numpy as np
import pandas as pd
import json

import os
import sys
from copy import deepcopy
import pdb
import pprint
import time

In [159]:
edie_report= {
  "high_aq_count": 0,
  "high_aq_set": [],
  "conversion_count": 0,
  "total_humans": 0,
  "live_valid_humans": 0,
  "live_humans": 0,
  "valid_humans": 0,
  "total_leaving": 0,
  "total_ignored": 0,
  "total_crossed_over": 0,
  "table_membership_duration": {
    "P_1": {
      "0":0.0
    },
    "P_0": {
      "0": 0.0
    }
  },
  "table_membership_high_aq": {
    "P_1": [],
    "P_0": []
  },
  "roi_staff": [],
  "attentive_staff": [],
  "attentive_interactions": [],
  "attentive_events": {},
  "attended_converted_count": 0,
  "attended_converted_set": [],
  "attended_set": [],
  "aq_converted_set": [],
  "customer_attention_log": {},
  "customer_conversion_log": {},
  "customer_entry_log": {
    "0": 0
  },
  "checkpoint_snapshot": None,
  "frame": 0
}


##Configure attributes for report

In [160]:
#Globals
# _P indicates values as percentage

TOT_HUMANS = 100
VALID_HUMANS_P = 0.8
TOT_STAFF = 5
ATTENTIVE_STAFF_P = 0.5
TOT_ATTENTIVE_STAFF_EVENTS = 6
ENTERED_CUSTOMERS_P = 0.2
SQ_EVENTS = 30
AQ_EVENTS = 30

CONVERSION_WITH_STAFF_ENGAGEMENT_P=0.3
CONVERSION_WITHOUT_STAFF_ENGAGEMENT_P = 1-CONVERSION_WITH_STAFF_ENGAGEMENT_P

TOP_STAFF=None
TOT_FRAMES = 143355

#
average_customer_traffic = 8

## Generate IDs for various categories of humans and entities in the report

In [161]:
shuffled_index_val = np.arange(0,2*TOT_HUMANS)
np.random.shuffle(shuffled_index_val)


tot_humans = shuffled_index_val[0:TOT_HUMANS]
tot_humans_set = set(tot_humans)
tot_frames = np.arange(0,TOT_FRAMES)


valid_human_count = int(VALID_HUMANS_P*TOT_HUMANS)
entered_customer_count = int(ENTERED_CUSTOMERS_P*valid_human_count)

valid_humans = shuffled_index_val[0:valid_human_count]
staff_id = shuffled_index_val[valid_human_count:valid_human_count+TOT_STAFF]
tot_attentive_staff = int(ATTENTIVE_STAFF_P*len(staff_id))
attentive_staff_id = staff_id[0:tot_attentive_staff]

entered_customers = valid_humans[0:entered_customer_count]

valid_humans_set = set(valid_humans)
entered_customer_set = set(entered_customers)
remaining_humans = tot_humans_set.difference(valid_humans_set)

## Generate various events between entities

### Create log of customer entry

In [162]:
#entered customers

tot_frame_index = np.arange(0,TOT_FRAMES)
shuffled_frame_index = deepcopy(tot_frame_index)
np.random.shuffle(shuffled_frame_index)
# if we need customer ids to be sorted
entered_customers = np.sort(entered_customers)

entered_customer_frames = shuffled_frame_index[0:entered_customer_count]
entered_customer_frames = np.sort(entered_customer_frames) # sort frames in ascending for chronological reasons

customer_entry_log = dict(zip(entered_customers, entered_customer_frames))

### Create log of staff-customer interactions

In [163]:
# attended events

dist_segments = len(attentive_staff_id)
dist_weights = np.random.randint(1,10,dist_segments)
dist_weights = np.float64(dist_weights)/sum(dist_weights)

attention_event_count = dist_weights*TOT_ATTENTIVE_STAFF_EVENTS
attention_event_count = (attention_event_count).astype(np.uint64)
attention_event_count[-1] = TOT_ATTENTIVE_STAFF_EVENTS - sum(attention_event_count[0:-1])
# print(attention_event_count)

attentive_event_log = {}
attended_list = []
for idx,val in enumerate(attentive_staff_id):
    attentive_event_log.update({val:{}})
    n_events = attention_event_count[idx]
    event_customers = np.random.choice(entered_customers, n_events, replace=True)
    event_frames = [np.random.randint(customer_entry_log[val], TOT_FRAMES) for val in event_customers]
    event_log = dict(zip(event_customers,event_frames))
    attentive_event_log.update({val:event_log})
    attended_list.append(list(event_customers.ravel()))

### Create list of attended customers
attended_list = list(map(list, zip(*staff_attention_set)))[1]

## Generate edie report based on config

In [164]:
edie_report.update({"frame":TOT_FRAMES})
edie_report.update({"valid_humans": valid_human_count})
edie_report.update({"total_humans":TOT_HUMANS})
edie_report.update({"roi_staff":list(staff_id)})
edie_report.update({"customer_entry_log":customer_entry_log})
edie_report.update({"attentive_events":attentive_event_log})
edie_report.update({"attended_set":set(sum(attended_list,[]))})
edie_report.update({"attentive_staff":set(attentive_staff_id)})

In [165]:
pprint.pprint(edie_report)

{'aq_converted_set': [],
 'attended_converted_count': 0,
 'attended_converted_set': [],
 'attended_set': {163, 37, 143, 15, 111},
 'attentive_events': {38: {15: 123883,
                           37: 48981,
                           111: 113295,
                           143: 102982,
                           163: 129201},
                      55: {}},
 'attentive_interactions': [],
 'attentive_staff': {38, 55},
 'checkpoint_snapshot': None,
 'conversion_count': 0,
 'customer_attention_log': {},
 'customer_conversion_log': {},
 'customer_entry_log': {15: 3955,
                        30: 16844,
                        37: 28005,
                        54: 29400,
                        63: 48436,
                        73: 53524,
                        80: 64015,
                        111: 64466,
                        124: 66622,
                        139: 70045,
                        143: 76814,
                        146: 88052,
                        154: 101583,
  

### Parameters for Chain-of-thought input

In [166]:
n = len(edie_report['customer_entry_log'])
s = len(edie_report['roi_staff'])
s1 = len(edie_report['attentive_staff'])
k = len(edie_report['attended_set'])

m = average_customer_traffic

print(f'n={n}\ns={s}\ns1={s1}\nk={k}')

n=16
s=5
s1=2
k=5


#### N-shot prompt generation

In [167]:
prompt_generator = {
    "How busy was the store today?":f'Looking at the cutomer_entry_log field, we see that there was a total of {n} customers that entered the store. Since {n} is higher than the average expected traffic of {m} customers, the store seems to have been busy.',
    "Were the customers properly attended to by the staff?":f'Looking at the customer_entry_log field, we see that there was a total of {n} customers at the store and looking at the attended_set, we see that {k} out of the {n} customers were attended to. So, {k} out of {n} customers were attended to which translates to roughly about {k*100.0/n} percentage of attended customers.',
    "Were the staff busy?":f'Looking at the roi_staff field, we see that there is a total of {s} staff. Further, looking at the attentive_staff field, we see that a total of {s1} staff attended to customers. This shows that {s1} out of {s} staff were busy.',
    "Did customers need a lot of help from staff?":f'Looking at the attentive_set field, we see that a total of {k} customers were attended to by staff. Looking at the customer_entry_log, we see that there were a total of {n} customers at the location. Since {k} out {n} customers were attended to, we can conclude that {k*100.0/n} percentage of customers needed help from staff.',
    }

In [170]:
for key,val in prompt_generator.items(): 
    print(f'q: {key}\nans: {val}\n\n')
    

q: How busy was the store today?
ans: Looking at the cutomer_entry_log field, we see that there was a total of 16 customers that entered the store. Since 16 is higher than the average expected traffic of 8 customers, the store seems to have been busy.


q: Were the customers properly attended to by the staff?
ans: Looking at the customer_entry_log field, we see that there was a total of 16 customers at the store and looking at the attended_set, we see that 5 out of the 16 customers were attended to. So, 5 out of 16 customers were attended to which translates to roughly about 31.25 percentage of attended customers.


q: Were the staff busy?
ans: Looking at the roi_staff field, we see that there is a total of 5 staff. Further, looking at the attentive_staff field, we see that a total of 2 staff attended to customers. This shows that 2 out of 5 staff were busy.


q: Did customers need a lot of help from staff?
ans: Looking at the attentive_set field, we see that a total of 5 customers wer